In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats, special

from tqdm.notebook import tqdm
import os
import itertools

In [94]:
from prophet import Prophet
from sklearn.linear_model import LogisticRegression

In [27]:
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [95]:
from optbinning import BinningProcess

In [257]:
from scipy.optimize import linprog

- Тестовый набора данных, для которых нужно сделать предсказание и составить расписание

In [68]:
test_data = pd.read_csv(r"test_data.csv")
test_data['date'] = pd.to_datetime(test_data['date'])

In [69]:
test_data.head()

,delivery_area_id,date,year,month,week,day_of_week,day,hour
0,0,2021-12-01 09:00:00,2021,12,48,2,1,9
1,0,2021-12-01 10:00:00,2021,12,48,2,1,10
2,0,2021-12-01 11:00:00,2021,12,48,2,1,11
3,0,2021-12-01 12:00:00,2021,12,48,2,1,12
4,0,2021-12-01 13:00:00,2021,12,48,2,1,13


- Orders modified, в нем заполненны пропуски часов работы зон доставки и добавленные некоторые признаки. Пропуски заполненные в соотвествии с расписанием работы каждой зоны доставки, так как они работают в разное время и разное кол-во часов 

In [7]:
data_orders = pd.read_csv(r"orders_modified.csv")
data_orders['date'] = pd.to_datetime(data_orders['date'])

In [8]:
data_orders.head()

,delivery_area_id,date,orders_cnt,year,month,week,day_of_week,day,hour
0,0,2021-04-01 09:00:00,0.0,2021,4,13,3,1,9
1,0,2021-04-01 10:00:00,9.0,2021,4,13,3,1,10
2,0,2021-04-01 11:00:00,1.0,2021,4,13,3,1,11
3,0,2021-04-01 12:00:00,0.0,2021,4,13,3,1,12
4,0,2021-04-01 13:00:00,1.0,2021,4,13,3,1,13


- Partners delays modified, добавленные данные о количестве заказов в соотвествии с зоной доставки и датой, также добавленные некоторые признаки

In [193]:
partners_delays = pd.read_csv(r"partners_delays_modified.csv")
partners_delays['dttm'] = pd.to_datetime(partners_delays['dttm'])

In [194]:
partners_delays.head()

,delivery_area_id,dttm,partners_cnt,delay_rate,orders_cnt,year,month,week,day_of_week,day,hour
0,0,2021-04-01 09:00:00,3.0,0.000000,0.0,2021,4,13,3,1,9
1,0,2021-04-01 10:00:00,4.0,0.111111,9.0,2021,4,13,3,1,10
2,0,2021-04-01 11:00:00,4.0,0.000000,1.0,2021,4,13,3,1,11
3,0,2021-04-01 12:00:00,4.0,0.000000,0.0,2021,4,13,3,1,12
4,0,2021-04-01 13:00:00,1.0,0.000000,1.0,2021,4,13,3,1,13


- Daily orders, количество заказов по дням

In [109]:
data_orders_daily= pd.read_csv(r"daily_orders.csv")
data_orders_daily['date'] = pd.to_datetime(data_orders_daily['date'])

In [110]:
data_orders_daily.head()

,delivery_area_id,date,orders_cnt,year,month,week,day
0,0,2021-04-01,24.0,2021,4,13,1
1,0,2021-04-02,21.0,2021,4,13,2
2,0,2021-04-03,33.0,2021,4,13,3
3,0,2021-04-04,18.0,2021,4,13,4
4,0,2021-04-05,32.0,2021,4,14,5


- Work hours by area, информация по времени работы каждой зоны доставки: кол-во часов, время начала и конца работы

In [255]:
work_hours= pd.read_csv(r"work_hours_by_areas.csv")

In [256]:
work_hours.head()

,delivery_area_id,hours,start,end,max_date
0,0,14,9,22,2021-11-30 22:00:00
1,1,14,8,21,2021-11-30 21:00:00
2,2,12,10,21,2021-11-30 21:00:00
3,3,13,9,21,2021-11-30 21:00:00
4,4,13,10,22,2021-11-30 22:00:00


**I ЭТАП**

Кратко:
- Для каждой Зоны доставки обучим Prophet
- Будем учить предсказывать общее кол-во заказов за день, а затем общее кол-во заказов за день распределим внутри дня по долям
- Используем только 9 последних недель для обучения, так как иначе Prophet плохо справлятся
- Зоны 2, 181, 182, 190, 195, 200, 367, 453 перестали функционировать, их предсказываем нулями

Последовательность действий:
1) Для ЗД #k берем данные
2) Преобразуем их box-cox
3) Обучаем Prophet
4) Делаем прогноз
5) Обратный box-cox
6) Распределяем кол-во заказов внутри дня

**Подсчет долей**

Подсчет долей от общего кол-ва заказов за день, распределение для каждой зоны доставки, каждого дня недели, по часам.
<br />Результатом будет словарь в виде словаря с ключами: (территория, день недели)
<br />Внутри будет словарь вида "час работы : доля"
<br />Результат для конкретного ключа: { ... ,14: 0.08, 15: 0.09, ...}

In [17]:
area_distribs = {}
data_orders_9_lastweeks = data_orders[data_orders.week>=40].copy()
groups = data_orders_9_lastweeks.groupby(by=['delivery_area_id','day_of_week'], dropna=False)
for vals, group in tqdm(groups):
    area = vals[0]
    day_of_week = vals[1]
    hour_groups = group.groupby(by=['hour'], dropna=False)
    area_distribs[(area, day_of_week)] = {}
    for hour, hour_group in hour_groups:
        parts=[]
        for i in hour_group.index:
            month =hour_group.loc[i,'month']
            day = hour_group.loc[i,'day']
            hour_count = hour_group.loc[i,'orders_cnt']
            day_count = data_orders_daily[(data_orders_daily.month == month) & (data_orders_daily.day == day) & (data_orders_daily.delivery_area_id == area)].orders_cnt.item()
            if day_count!=0:
                parts.append(hour_count/day_count)
        if len(parts)!=0:
            median = np.median(np.array(parts))
            area_distribs[(area, day_of_week)][hour] = median
        else:
            area_distribs[(area, day_of_week)][hour] = 0

  0%|          | 0/4151 [00:00<?, ?it/s]

In [18]:
area_distribs[(0, 5)]

{9: 0.0,
 10: 0.11672413793103448,
 11: 0.09084745762711866,
 12: 0.09187783185098171,
 13: 0.10261016949152543,
 14: 0.09461016949152543,
 15: 0.08787991104521868,
 16: 0.08197684175197181,
 17: 0.07413559322033898,
 18: 0.06418572891771936,
 19: 0.09710526315789474,
 20: 0.06589830508474576,
 21: 0.020563380281690143,
 22: 0.0}

**Прогнозирование**

In [24]:
data_orders_daily_9_weeks = data_orders_daily[data_orders_daily.week>=40].copy()
data_orders_daily_9_weeks['ds'] = data_orders_daily_9_weeks['date']
data_orders_daily_9_weeks['y'] = data_orders_daily_9_weeks['orders_cnt']

In [86]:
bad_areas = {2, 453, 195, 190, 200, 367, 181, 182}
areas = set(data_orders_daily.delivery_area_id.unique()) - bad_areas
lst = []
for area in tqdm(areas):
    
    # Береме данные зоны доставки
    area_data = data_orders_daily_9_weeks[data_orders_daily_9_weeks.delivery_area_id == area].copy()
    
    # Box-cox проебразование
    try:
        area_data['y'], lmbda = stats.boxcox(area_data['orders_cnt'])
    except:
        area_data['orders_cnt'] += 1
        area_data['y'], lmbda = stats.boxcox(area_data['orders_cnt'])
    
    # Обучаем Prophet
    model = Prophet(yearly_seasonality=False, daily_seasonality=False, weekly_seasonality=10, seasonality_mode='multiplicative')
    model.fit(area_data)
    
    # Делаем прогноз
    # Индекс 0 - ds, 15 - yhat
    dates = model.make_future_dataframe(periods=7)
    test_daily_preds = model.predict(dates).iloc[-7:,[0,15]].copy()
    
    # Обратный Box-cox
    test_daily_preds['ordershat'] = np.round(special.inv_boxcox(test_daily_preds['yhat'], lmbda))
    
    # Распределение кол-ва заказов внутри дня
    for date in test_daily_preds.ds:
        daily_ord_count = test_daily_preds[test_daily_preds.ds == date].ordershat.item()
        day_of_week = date.weekday()
        month = date.month
        day = date.day
        intraday_distribution = area_distribs[(area, day_of_week)]
        test_data_part = test_data[(test_data.delivery_area_id == area) & (test_data.month == month) & (test_data.day == day)].copy()
        test_data_part['orders_cnt'] = daily_ord_count * np.fromiter(intraday_distribution.values(), dtype=float)
        lst.append(test_data_part)

# Заполнение нулями зон, переставших функционировать
for area in tqdm(bad_areas):
    test_data_part = test_data[test_data.delivery_area_id == area].copy()
    test_data_part['orders_cnt'] = 0
    lst.append(test_data_part)
    
test_data = pd.concat(lst)

  0%|          | 0/585 [00:00<?, ?it/s]

INFO:prophet:n_changepoints greater than number of observations. Using 21.
INFO:prophet:n_changepoints greater than number of observations. Using 21.


  0%|          | 0/8 [00:00<?, ?it/s]

In [189]:
test_data['orders_cnt'] = np.round(test_data['orders_cnt'])

In [192]:
test_data.head()

,delivery_area_id,date,year,month,week,day_of_week,day,hour,orders_cnt
0,0,2021-12-01 09:00:00,2021,12,48,2,1,9,0.0
1,0,2021-12-01 10:00:00,2021,12,48,2,1,10,14.0
2,0,2021-12-01 11:00:00,2021,12,48,2,1,11,11.0
3,0,2021-12-01 12:00:00,2021,12,48,2,1,12,9.0
4,0,2021-12-01 13:00:00,2021,12,48,2,1,13,10.0


**II Этап**

Кратко:
- Решим задачу бинарной классификации, для этого создадим бинарный признак `high_delay_rate`, равный 1 если `delay_rate >= 0.05` и равный 0 иначе
- Избавимся от категориальных признаков (`delivery_area_id`, `month`, `hour`, `day_of_week`), воспользовавшись WoE бинингом признаков, это приводит их в формат более удобный для подачи в логистическую регрессию
- Обучаем логистическую регрессию (в ходе тестов выбор пал на threshold = 0.29)
- После для каждой строчки из тестового набора данных находим минимальное кол-во курьеров, чтобы наша логистическая регрессия выдалава 0, то бишь `delay_rate<0.05`

In [197]:
partners_delays['high_delay_rate'] = partners_delays['delay_rate'].apply(lambda x: 1 if x > 0.05 else 0)
partners_delays['ords_per_partner'] = partners_delays['orders_cnt'] / partners_delays['partners_cnt']

WoE биннинг

In [198]:
def feature_informatives(data, target_feature, features_for_check, n=10):
    '''
    data - данные
    target_feature - название целевой признака
    features_for_check - массив признаков для проверки
    n - кол-во строк в возвращаемом датафрейме
    
    Функция возвращает датафрейм из n признаков с наибольшим IV
    '''
    
    columns=['Признак','IV','Интерпретация IV']
    lst=[]
    
    y=data[target_feature]
    x=data[features_for_check]
    optb_proc = BinningProcess(variable_names=features_for_check, n_jobs=-1).fit(x, y)
    for feature in features_for_check:
        optb_table = optb_proc.get_binned_variable(feature).binning_table.build()
        IV = optb_table.loc['Totals','IV']
        if IV<0.02:
            intpr = 'Бесполезен для предсказаний'
        elif IV<0.1:
            intpr = 'Слабая предсказательная сила'
        elif IV<0.3:
            intpr = 'Средняя предсказательная сила'
        elif IV<0.5:
            intpr = 'Сильная предсказательная сила'
        else:
            intpr = 'Подозрительно хорош'
        lst.append([feature, IV, intpr])
    df = pd.DataFrame(columns=columns, data=lst)
    return df.sort_values(by='IV', ascending=False).head(n)

Интересное набллюдение, что `delivery_area_id` обладает сильной предсказательной силой, в некоторых интервалах номеров зон доставок попадает большее ко-во случаев с `delay_rate>=0.05`

In [199]:
features_for_binning = ['delivery_area_id','partners_cnt','orders_cnt','month','day_of_week','hour','ords_per_partner']
res = feature_informatives(partners_delays, 'high_delay_rate', features_for_binning, n=100)
res

,Признак,IV,Интерпретация IV
2,orders_cnt,2.832166,Подозрительно хорош
6,ords_per_partner,2.343066,Подозрительно хорош
1,partners_cnt,1.906699,Подозрительно хорош
0,delivery_area_id,0.365996,Сильная предсказательная сила
3,month,0.236002,Средняя предсказательная сила
5,hour,0.090543,Слабая предсказательная сила
4,day_of_week,0.004686,Бесполезен для предсказаний


Кодирование признаков

In [201]:
optb_proc = BinningProcess(variable_names=features_for_binning, n_jobs=-1).fit(partners_delays[features_for_binning] , partners_delays['high_delay_rate'])
features = []
for feature in features_for_binning:
    optb = optb_proc.get_binned_variable(feature)
    partners_delays.loc[:,f'{feature}_woe'] = optb.transform(partners_delays[feature], metric="woe")
    features.append(f'{feature}_woe')

In [202]:
partners_delays.head()

,delivery_area_id,dttm,partners_cnt,delay_rate,orders_cnt,year,month,week,day_of_week,day,hour,high_delay_rate,ords_per_partner,delivery_area_id_woe,partners_cnt_woe,orders_cnt_woe,month_woe,day_of_week_woe,hour_woe,ords_per_partner_woe
0,0,2021-04-01 09:00:00,3.0,0.000000,0.0,2021,4,13,3,1,9,0,0.00,-1.060353,-0.877442,2.703654,0.239222,0.011375,0.018281,2.965516
1,0,2021-04-01 10:00:00,4.0,0.111111,9.0,2021,4,13,3,1,10,1,2.25,-1.060353,-1.373419,-1.800556,0.239222,0.011375,0.018281,-2.288671
2,0,2021-04-01 11:00:00,4.0,0.000000,1.0,2021,4,13,3,1,11,0,0.25,-1.060353,-1.373419,2.703654,0.239222,0.011375,0.018281,2.965516
3,0,2021-04-01 12:00:00,4.0,0.000000,0.0,2021,4,13,3,1,12,0,0.00,-1.060353,-1.373419,2.703654,0.239222,0.011375,0.018281,2.965516
4,0,2021-04-01 13:00:00,1.0,0.000000,1.0,2021,4,13,3,1,13,0,1.00,-1.060353,2.120894,2.703654,0.239222,0.011375,0.018281,1.465965


Обучение Логистической регрессии

In [203]:
featurs = ['delivery_area_id_woe','partners_cnt_woe','orders_cnt_woe','month_woe','day_of_week_woe','hour_woe','ords_per_partner_woe']
logreg = LogisticRegression(penalty = 'l2', random_state=42).fit(partners_delays[features], partners_delays['high_delay_rate'])

Поиск минимального кол-ва курьеров для каждой строчки тестового датасета

In [214]:
lst = []

for i in tqdm(test_data.index):
    
    row = test_data.loc[[i],:].copy()
    is_delay_rate_low = 0
    partners_cnt = 1
    
    while is_delay_rate_low == 0:
        
        row['partners_cnt'] = partners_cnt
        row['ords_per_partner'] = row['orders_cnt']/row['partners_cnt']
        
        new_row = optb_proc.transform(row)
        prob = logreg.predict_proba(new_row)[:, 1][0]
        
        if prob < 0.29:
            is_delay_rate_low = 1
            lst.append(row)
        else:
            partners_cnt += 1
            
test_data = pd.concat(lst)

  0%|          | 0/51751 [00:00<?, ?it/s]

**III Этап**

Воспользуемся симлекс методом

Функция для создания матрицы А (все возможные смены)

In [246]:
def get_A(n):
    '''
    n - кол-во часов работы
    '''
    hours_4 = np.array([i*[0] + 4*[1] + (n-4-i)*[0] for i in range(n-4+1)])
    hours_5 = np.array([i*[0] + 5*[1] + (n-5-i)*[0] for i in range(n-5+1)])
    hours_6 = np.array([i*[0] + 6*[1] + (n-6-i)*[0] for i in range(n-6+1)])
    hours_7 = np.array([i*[0] + 7*[1] + (n-7-i)*[0] for i in range(n-7+1)])
    hours_8 = np.array([i*[0] + 8*[1] + (n-8-i)*[0] for i in range(n-8+1)])
    return np.concatenate((hours_4,hours_5,hours_6,hours_7,hours_8))

Создание расписания

In [ ]:
groups = test_data.groupby(by=['delivery_area_id','month','day'], dropna=False)

In [354]:
cols = ['delivery_area_id', 'date', '№' ,'work']
lst = []
current_area = -1
for vals, group in groups:
    area = vals[0]
    month = vals[1]
    day = vals[2]
    if area != current_area:
        
        current_area = area
        hours_count = work_hours[work_hours.delivery_area_id == area].hours.item()
        start_hour = work_hours[work_hours.delivery_area_id == area].start.item()
        end_hour = work_hours[work_hours.delivery_area_id == area].end.item()
        hours = np.arange(start_hour, end_hour+1)
        
        A = get_A(hours_count)
        c = np.array([1 for i in range(A.shape[0])])
        
    h = np.array(group.partners_cnt)
    optimize_result = linprog(c, A_ub= -A.T, b_ub= -h, method = 'highs')
    x = optimize_result.x
    
    idx = np.arange(len(x))
    non_zero_idx = idx[x[idx]>0]
    for k, i in enumerate(non_zero_idx):
        smena = A[i]
        interval_start = min(hours[smena>0])
        interval_end = max(hours[smena>0]) + 1
        #work_str += f'{interval_start}:00 - {interval_end}:00 {int(x[i])} чел., '
        lst.append([area, pd.Timestamp(f'2021-{month}-{day}'), k+1 ,f'{int(x[i])} человек на смену {interval_start}:00 - {interval_end}:00'])
    #work_str = work_str[:-2]
    #lst.append([area, pd.Timestamp(f'2021-{month}-{day}'), work_str])
timetable = pd.DataFrame(data = lst, columns = cols)

In [355]:
timetable.head(10)

,delivery_area_id,date,№,work
0,0,2021-12-01,1,2 человек на смену 10:00 - 14:00
1,0,2021-12-01,2,13 человек на смену 9:00 - 16:00
2,0,2021-12-01,3,12 человек на смену 15:00 - 23:00
3,0,2021-12-02,1,11 человек на смену 10:00 - 14:00
4,0,2021-12-02,2,4 человек на смену 14:00 - 20:00
5,0,2021-12-02,3,5 человек на смену 9:00 - 16:00
6,0,2021-12-02,4,11 человек на смену 15:00 - 23:00
7,0,2021-12-03,1,9 человек на смену 10:00 - 14:00
8,0,2021-12-03,2,13 человек на смену 9:00 - 16:00
9,0,2021-12-03,3,15 человек на смену 15:00 - 23:00
